## 使用python3实现mysql导出数据到文件的脚本

##### mysql分页直接用limit start, count分页语句：
select * from product limit start, count  从start开始分页,每页取count条

In [1]:
# 导入pymysql模块
import pymysql
import time 
import math
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("expand.frame_repr",False)
print('import succed!')

import succed!


In [2]:
# 建立连接——连接database数据库
conn = pymysql.connect(
    host="114.215.123.44", 
    port =3306, 
    user="root", 
    password="W01fstudio",
    charset="utf8")

In [3]:
# 获取光标——得到一个可以执行SQL语句的光标对象
cursor = conn.cursor()

In [4]:
# 统计总的条目数
sql1 = "select fromcheckid,sendreadtime,casehospitalname,readhospitalname from zn_ai.d_baseinfo;" # 输出数据库zn_ai.d_baseinfo表包含的总数据量
print("欲查询的Table包含——{}条数据".format(cursor.execute(sql1)))

欲查询的Table包含——256条数据


In [5]:
def select_fuc(sql_x, limit_num, count_scale_x):
    df_last = pd.DataFrame()
    for i in range(count_scale_x):    # range(n) —— 表示循环 n 次 
        start_num = 0 + limit_num*i    # 导出数据的开始索引
        end_num = limit_num * (i+1)   # 导出数据的末尾索引

        start_num_i = str(start_num)   # 起始数据索引——转为字符形式str
        end_num_i = str(end_num)       # 末尾数据索引——转为字符形式str
        limit_num_i = str(limit_num) 
#         print("起始索引_{0},终止索引_{1}".format(start_num_i,end_num_i))
        sql = sql_x + start_num_i + "," + limit_num_i + ";"
        cursor.execute(sql)      # 执行SQL语句
        data  = cursor.fetchall()  # 使用fetchone()方法创建一个游标对象   
    #     print(data)
    #     print("="*50)
        df_last = df_last.append(pd.DataFrame(data),ignore_index=True)   # 添加数据集子  ignore_index=True表示重排列索引
#         print(df_last)
    print("合并后的数据总量：{}".format(df_last.shape[0]))
        
    return df_last

# 设置参数
limit_num = 50   # 数据规格大小
sql_x = "select fromcheckid,sendreadtime,casehospitalname,readhospitalname from zn_ai.d_baseinfo limit " 
df_temp01 = select_fuc(sql_x,limit_num)  # 调用查询数据函数，并写入定义的空DataFrame中，用于存放查询出来的数据
df_temp01.columns = ['fromcheckid','sendreadtime','casehospitalname','readhospitalname']    # 重命名列名

#### （1）向上取整：math.ceil()——严格遵循向上取整，所有小数都是向着数值更大的方向取整，不论正负数都如此
#### （2）四舍五入：round()——round()当不传入第二个参数时默认取整，具体就是按照四舍五入来。但值得一提的是这里对小数末尾为5的处理方法：当末尾的5的前一位为奇数：向绝对值更大的方向取整（比如-1.5、1.5处理结果）；当末尾的5的前一位为偶数：去尾取整（比如-2.5，-0.5，0.5和2.5的处理结果）。
例如：round(1.5) >>> 2      round(2.5)  >>> 2
#### （3）向下取整：math.floor()——简单且忠实地向下取整
#### （4）两个有趣且特殊的Python取整：int()、整除"//"
一句话总结：int()函数是“向0取整”，取整方向总是让结果比小数的绝对值更小；一句话总结：“整除”符号运算将结果忠实地向下取整，与math.floor()处理结果一样

In [10]:
def output_fuc(table_items,Table_name,limit_num):
    for j in range(table_items):
        # 统计总的条目数
        table_number = 0   # 初始表的索引值
        # Table_name = "d_checkimage_" + str(table_number)  # 准备导出的表名称 ————————最终使用
        print("欲导出数据表的名称: {}".format(Table_name))
        sql_count = "select fromcheckid,sendreadtime,casehospitalname,readhospitalname from zn_ai." + Table_name + "; "# 输出数据库zn_ai.d_baseinfo表包含的总数据量
        table_x_count = cursor.execute(sql_count)  # 统计表_x的容量大小
        print("欲查询的Table包含——{}条数据".format(table_x_count))
        limit_num = 50    # 导出数据的规格大小
        count_scale = math.ceil(table_x_count/limit_num)   # 数据分段大小，math.ceil()向上取整
        print("原始数据集将被分割为: {} 段".format(count_scale))
        # 定义导出SQL
        sql_x = "select fromcheckid,sendreadtime,casehospitalname,readhospitalname from zn_ai." + Table_name + " limit "  
        table_number += j # 表名称递增
        ## 函数调用+整理
        df_temp = select_fuc(sql_x, limit_num, count_scale)   # 调用函数
        df_temp.columns = ["fromcheckid","sendreadtime","casehospitalname","readhospitalname"]   # 重定义列名
        ## 校验
        a_x = df_temp[df_temp['fromcheckid'].duplicated()==True]
        if a_x.shape[0] == 0:
            print("表——X无重复数据项，结果正确！！！！！")
        else:
            print("表——X有重复数据项，结果有误！！！！！")
            break
        ## 写出数据
        houzui = str(j)
        output_path = r"/Users/edz/Documents/yandi_data/AI_data/zd_dc"
        output_file = r"/d_checkimage_" + houzui + ".csv"
        df_temp.to_csv(output_path + output_file,index=None)

## 函数调用+整理

In [11]:
Table_name_a = "d_baseinfo"  # 准备导出的表名称————————暂时使用
limit_num_a = 50    # 导出数据的规格大小
table_items_a = 16
output_fuc(table_items_a,Table_name_a,limit_num_a)

欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据项，结果正确！！！！！
欲导出数据表的名称: d_baseinfo
欲查询的Table包含——256条数据
原始数据集将被分割为: 6 段
合并后的数据总量：256
表——X无重复数据

In [ ]:
# sql2 = "select * from zn_ai.d_baseinfo where fromcheckid!='' and fromcheckid is not null;"

In [ ]:
# 关闭光标对象
cursor.close()

In [ ]:
# 关闭数据库连接
conn.close()